In [ ]:
!pip install einops timm

In [ ]:
class SoftCrossEntropyLoss():
    def __init__(self, weights):
        super().__init__()
        self.weights = weights

def forward(self, y_hat, y):
    p = F.log_softmax(y_hat, 1)
    w_labels = self.weights*y
    loss = -(w_labels*p).sum() / (w_labels).sum()
    return loss

x = torch.randn(1,3)
x

In [ ]:
import torch
import torch.nn as nn
from einops import rearrange, repeat
# from lambda_networks import LambdaLayer
from torch import einsum
# make required import
import torch 
import timm 
import wandb 
import os
import torchvision
import numpy as np 
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from accelerate import Accelerator
from PIL import Image 
from matplotlib import pyplot as plt
from pathlib import Path 
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from dataclasses import dataclass
from timm.models.registry import register_model
from timm.models.helpers import build_model_with_cfg
from timm.models.resnet import Bottleneck, _create_resnet, default_cfgs, _cfg, make_blocks, create_classifier
import torch
import torch.nn as nn
from einops import rearrange, repeat
# from lambda_networks import LambdaLayer
from torch import einsum
# from lambda_networks import LambdaLayer

# https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/layers/common_attention.py
def get_relative_position_matrix(size,):
    x = torch.repeat_interleave(torch.arange(size),size,dim=0)
    y = torch.arange(size).repeat(size)
    distance_mat = torch.stack((x,y))
    distance_mat = distance_mat[:,None, :] - distance_mat[:, :,None]
    distance_mat = torch.clamp(distance_mat, -size, size)
    distance_mat += size-1
    return distance_mat

class LambdaLayer_own(nn.Module):
    def __init__(self, dim, dim_k=16,n=64, dim_out=None, heads=4, r=None):
        super().__init__()
        self.dim_out = dim_out
        self.r = r
        self.n = n
        self.k = dim_k
        assert(dim_out % heads) == 0
        self.heads = heads
        self.dim_v = dim_out // heads

# (Page 5) BN after Q and V are helpful
        self.get_q = nn.Sequential(
            nn.Conv2d(in_channels=dim, out_channels=dim_k*heads, kernel_size=1, bias=False),
            nn.BatchNorm2d(dim_k*heads),
        )
        self.get_v = nn.Sequential(
            nn.Conv2d(in_channels=dim, out_channels=self.dim_v, kernel_size=1, bias=False),
            nn.BatchNorm2d(self.dim_v),
        )
        self.get_k = nn.Conv2d(in_channels=dim, out_channels=dim_k, kernel_size=1, bias=False)

        if r is not None:
            assert (r % 2) == 1, 'Receptive kernel size should be odd'
            self.pos_conv = nn.Conv3d(1, self.k, (r, r, 1), padding=(r // 2, r // 2, 0))
        else:   
            # rel_lengths = 2 * n - 1 # n = im_h = im_w the feature map size
            # self.rel_pos_emb = nn.Parameter(torch.randn(rel_lengths, rel_lengths, dim_k)) # n m k 
            self.rel_pos = get_relative_position_matrix(n)
            rel_lengths = 2 * self.n - 1 # n = im_h = im_w the feature map size
            self.rel_pos_emb = nn.Parameter(torch.randn(rel_lengths, rel_lengths, self.k)) # 2*n-1 2*n-1 k 

    def forward(self,x):
        (b, c, im_h, im_w) = x.shape
        Q = self.get_q(x)
        K = self.get_k(x)
        V = self.get_v(x)

        # m = n = im_h * im_w
        Q = rearrange(Q, 'b (h k) im_h im_w -> b h (im_h im_w) k', h=self.heads) # b h n(m) k 
        K = rearrange(K, 'b k im_h im_w -> b (im_h im_w) k') # b m k 
        V = rearrange(V, 'b v im_h im_w -> b (im_h im_w) v') # b m v

        σ_K = K.softmax(dim=-1)
        λc = einsum('b m k, b m v -> b k v', σ_K, V)
        content_output = einsum('b h n k, b k v -> b n h v', Q, λc)

        if self.r is not None:
            V = rearrange(V, 'b (im_h im_w) v -> b v im_h im_w', im_h = im_h, im_w = im_w).reshape(b, 1, self.dim_v, im_h, im_w,)
            embeddings = self.pos_conv(V)
            position_output = einsum('b h n k, b k v n -> b n h v', Q, embeddings.flatten(3))
        else:
            # embeddings = get_embedding(self.k, im_h) # n m k
            embeddings = self.rel_pos_emb[self.rel_pos[0], self.rel_pos[1]]
#             print(embeddings.shape)
#             print(V.shape)
            λp = einsum('n m k, b m v -> b n k v', embeddings, V)
            position_output = einsum('b h n k, b n k v -> b n h v',Q, λp)

        output = rearrange(content_output + position_output, 'b (im_h im_w) h v -> b (h v) im_h im_w', im_h=im_h, im_w=im_w)

        return output


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None,feature_size=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        # self.conv2 = conv3x3(planes, planes)
#         self.conv2 = LambdaLayer(
#                             dim = planes,
#                             dim_out = planes,
#                             n = feature_size,
# #                             r = 23,         # the receptive field for relative positional encoding (23 x 23)
#                             dim_k = 16,
#                             heads = 4,
#                             dim_u = 4
#                         )
        self.conv2 = LambdaLayer_own(dim=planes, dim_out=planes, n=feature_size)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class BasicBlock_Rs(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None,feature_size=None):
        super(BasicBlock_Rs, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        # self.conv2 = LambdaLayer(dim=planes, dim_out=planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

    

class ResNet(nn.Module):

    def __init__(self, block_lambda, block_rs, layers, num_classes=10, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None,):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        # self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1,
        #                        bias=False)
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block_rs, 64, layers[0], feature_size=56)
        self.layer2 = self._make_layer(block_lambda, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0],feature_size=28) # 56/2=
        self.layer3 = self._make_layer(block_rs, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1],feature_size=14) # 14
        self.layer4 = self._make_layer(block_rs, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2],feature_size=7) # 7
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block_rs.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False, feature_size=None):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer, feature_size=feature_size))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):

            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer,feature_size=feature_size))


        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block,BasicBlock_Rs, layers, pretrained, progress, **kwargs):
    model = ResNet(block, BasicBlock_Rs, layers, **kwargs)
    # if pretrained:
    #     state_dict = load_state_dict_from_url(model_urls[arch],
    #                                           progress=progress)
    #     model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock,BasicBlock_Rs, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)

# [2, 2, 2, 2] RL-26
# [2, 3, 5, 2] RL-38
# [3, 4, 6, 3] RL-50

def resnet34(pretrained=False, progress=True, **kwargs):
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', BasicBlock,BasicBlock_Rs, [2, 3, 5, 2], pretrained, progress,
                   **kwargs)



def resnet50(pretrained=False, progress=True, **kwargs):
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', BasicBlock, BasicBlock_Rs,[3, 4, 6, 3], pretrained, progress,
                   **kwargs)


class ImageNette(Dataset):
    
    def __init__(self, csv_file, root_dir, noisy_level=0, transform=None, train=True):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.image_path_list = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.noisy_level = noisy_level
        self.train = train
        self.train_map = {
             'n02115641': 0,
             'n02086240': 1,
             'n02088364': 2,
             'n02087394': 3,
             'n02105641': 4,
             'n02111889': 5,
             'n02099601': 6,
             'n02096294': 7,
             'n02093754': 8,
             'n02089973': 9
        }
#         { ImageNette
#             'n02979186': 0,
#             'n03417042': 1,
#             'n01440764': 2,
#             'n02102040': 3,
#             'n03028079': 4,
#             'n03888257': 5,
#             'n03394916': 6,
#             'n03000684': 7,
#             'n03445777': 8,
#             'n03425413': 9
#         }
        self.val_map ={
             'n02115641': 0,
             'n02086240': 1,
             'n02088364': 2,
             'n02087394': 3,
             'n02105641': 4,
             'n02111889': 5,
             'n02099601': 6,
             'n02096294': 7,
             'n02093754': 8,
             'n02089973': 9
        } 
    
#     { ImageNette
#             'n02979186': 0,
#             'n03417042': 1,
#             'n01440764': 2,
#             'n02102040': 3,
#             'n03028079': 4,
#             'n03888257': 5,
#             'n03394916': 6,
#             'n03000684': 7,
#             'n03445777': 8,
#             'n03425413': 9
#         }


    def __len__(self):
        return len(self.image_path_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.root_dir,
                                self.image_path_list['path'][idx])
        image = Image.open(img_path).convert('RGB')
        labels_head = 'noisy_labels_'+str(self.noisy_level)
        if self.train:
            label = self.train_map[self.image_path_list[labels_head][idx]]
        else:
            label = self.val_map[self.image_path_list[labels_head][idx]]
        label = torch.from_numpy(np.array([label]))

        if self.transform:
#             try:
            image = self.transform(image)
#             except:
#                 print(img_path)
        sample =  [image, label]
        return sample


# Config for training 
IMG_SIZE = 224

Config = dict(
    DATA_DIR="../input/imagenette/imagenette",
    TRAIN_DATA_DIR="../input/imagenette/imagenette/train",
    TEST_DATA_DIR="../input/imagenette/imagenette/val",
    DEVICE="cuda",
    PRETRAINED=False,
    LR=1e-5,
    EPOCHS=50,
    IMG_SIZE=IMG_SIZE,
    BS=64,
    TRAIN_AUG=transforms.Compose(
        [
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.RandomErasing(0.2),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ]
    ),
    TEST_AUG=transforms.Compose(
        [
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ]
    ),
)

# image_path_label = pd.read_csv(csv_file)
# train_path_label = image_path_label.loc[image_path_label['is_valid'] == 0]
# val_path_label = image_path_label.loc[image_path_label['is_valid'] == 1]

def train_fn(model, train_data_loader, optimizer, epoch, accelerator,scheduler):
    model.train()
    fin_loss = 0.0
    tk = tqdm(train_data_loader, desc="Epoch" + " [TRAIN] " + str(epoch + 1))

    for t, data in enumerate(tk):
        optimizer.zero_grad()
        out = model(data[0])
#         print(data[1].shape)
        loss = nn.CrossEntropyLoss()(
            out, data[1].flatten()
            )
        accelerator.backward(loss)
        optimizer.step()
        scheduler.step(epoch + t / len(train_data_loader))
        fin_loss += loss.item()
        tk.set_postfix(
            {
                "loss": "%.6f" % float(fin_loss / (t + 1)),
                "LR": optimizer.param_groups[0]["lr"],
            }
        )
    return fin_loss/len(train_data_loader), optimizer.param_groups[0]["lr"]

def eval_fn(model, eval_data_loader, epoch):
    model.eval()
    fin_loss = 0.0
    tk = tqdm(eval_data_loader, desc="Epoch" + " [VALID] " + str(epoch + 1))

    with torch.no_grad():
        for t, data in enumerate(tk):
            out = model(data[0])
            loss = nn.CrossEntropyLoss()(
                out, data[1].flatten()
                )
            fin_loss += loss.item()
            tk.set_postfix({"loss": "%.6f" % float(fin_loss / (t + 1))})
        return fin_loss/len(eval_data_loader)

def train():
    accelerator = Accelerator()

    # wandb inita
    wandb.init(config=Config, project='imagewoof', save_code=True, 
           job_type='train', tags=['resnetrs', 'imagewoof'], 
           name='LambdaR18_25_lambda2_r_None')    
    
    # train and eval datasets 
#     train_dataset = torchvision.datasets.ImageFolder(
#         Config['TRAIN_DATA_DIR'], 
#         transform=Config['TRAIN_AUG']
#         )
#     eval_dataset = torchvision.datasets.ImageFolder(
#         Config['TEST_DATA_DIR'], 
#         transform=Config['TEST_AUG']
#         )


    train_dataset = ImageNette(
#         csv_file='../input/imagenette/imagenette/train_noisy_imagenette.csv', 
        csv_file = '../input/imagewoof/imagewoof2/train_noisy_imagewoof.csv',
#         root_dir='../input/imagenette/imagenette', 
        root_dir = '../input/imagewoof/imagewoof2',
        noisy_level=25, # 0 1 5 25 50
        transform=Config['TRAIN_AUG'], 
        train=True 
    )
    eval_dataset = ImageNette(
#         csv_file='../input/imagenette/imagenette/val_noisy_imagenette.csv', 
#         root_dir='../input/imagenette/imagenette', 
        csv_file = '../input/imagewoof/imagewoof2/val_noisy_imagewoof.csv',
        root_dir = '../input/imagewoof/imagewoof2',
        noisy_level=0, 
        transform=Config['TEST_AUG'], 
        train=False
    )

    # train and eval dataloaders
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=Config["BS"]
    )
    eval_dataloader = torch.utils.data.DataLoader(
        eval_dataset, batch_size=Config["BS"]
    )

    # model
    model = resnet34().cuda()

    # optimizer    
    optimizer = torch.optim.Adam(
        model.parameters(), lr=Config["LR"]
    )
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2, eta_min=0)
    # prepare for DDP
    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)    

    for epoch in range(Config["EPOCHS"]):
        avg_loss_train, lr = train_fn(
            model, train_dataloader, optimizer, epoch, accelerator,scheduler)
        avg_loss_eval = eval_fn(
            model, eval_dataloader, epoch)
        wandb.log({'train_loss': avg_loss_train, 'eval_loss': avg_loss_eval, 'lr': lr})
    torch.save(model.state_dict(), 'LambdaR18_25_lambda2_r_None.pt')
        
train()